<a href="https://colab.research.google.com/github/lydiahiba/exploradome_tangram/blob/cnn---team-2/projet_exploradome_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pour commencer installer le weight and baias (wandb)

In [ ]:
!pip install wandb

In [ ]:
import os
import math
import datetime
from fastai.vision import *
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import wandb
from wandb.keras import WandbCallback

## Hyperparams

In [ ]:
# hyper params
# Set up your default hyperparameters before wandb.init
# so they get properly set in the sweep
hyperparameter_defaults = dict(
    dropout = 0.25,
    channels_one = 32,
    channels_two = 32,
    batch_size = 32,
    decay = 0.0, 
    earlyStopPatience = 10,
    trainRatio = 0.8,
    beta_1 = 0.9,
    beta_2 = 0.999,
    dense_layer_nodes = 256,
    learning_rate = 0.01/3,
    epochs = 50,
    #optimizer = tf.optimizers.Adamax(lr = config.learning_rate),
    splitt = [0.2, 0,3], 
    activation_function = ['softmax', 'softplus', 'softsign', 'tanh' ],
    size_img = 250
  
    )

# Pass your defaults to wandb.init
wandb.init(config=hyperparameter_defaults)
config = wandb.config


# Log metrics inside your training loop
# metrics = {'accuracy': accuracy, 'loss': loss}
# wandb.log(metrics)

## Connect in google drive

In [ ]:
# Google drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

## Unzip file

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# creata directory videos
# os.makedirs("data", exist_ok=True)
# !mv data /content

In [ ]:
!ls content

In [ ]:
# !unzip "/content/gdrive/My Drive/Exploradome/Dataset/Train-set.zip" -d "/content/train"

In [ ]:
# !unzip "/content/gdrive/My Drive/Exploradome/Dataset/Valid_images.zip" -d "/content/val"

In [ ]:
# !unzip "/content/gdrive/My Drive/Exploradome/Dataset/Test.zip" -d "/content"

In [ ]:
!ls content

In [ ]:
os.makedirs('val3',exist_ok=True)
!mv val3 /content 
!unzip "/content/gdrive/My Drive/Exploradome Project /Copie de Valid_images.zip" -d "/content/val3"

## Create labels

In [ ]:
# create labels and path
TRAIN = ("/content/gdrive/My Drive/Dataset-gp2/Train-set")
VAL = ("/content/gdrive/My Drive/Dataset-gp2/Valid-set")
TEST = ("/content/gdrive/My Drive/Dataset-gp2/Test-set")
VAL3=("/content/val3")
Labels = ['Bateau', 'Bol', 'Chat', 'Coeur', 'Cygne', 'Lapin', 'Maison', 'Marteau', 'Montagne', 'Pont', 'Renard','Tortue']

# Image per label

Train

In [ ]:
for i in os.listdir("/content/gdrive/My Drive/Dataset-gp2/Train-set"):
 print("/content/gdrive/My Drive/Dataset-gp2/Train-set"+"/"+i)
 print(len(os.listdir("/content/gdrive/My Drive/Dataset-gp2/Train-set"+"/"+i))) 

In [ ]:
for i in os.listdir("/content/gdrive/My Drive/Dataset-gp2/Train-set"):
 print("/content/gdrive/My Drive/Dataset-gp2/Train-set"+"/"+i)
 print(len(os.listdir("/content/gdrive/My Drive/Dataset-gp2/Train-set"+"/"+i)))

In [ ]:
# Show img train
for label in Labels:
    path = os.path.join(TRAIN, label)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        print(os.path.join(path, img))
        plt.imshow(img_array, cmap="gray")
        plt.title(Labels[os.listdir(path).index(img)])
        plt.show()
        break
    break

In [ ]:
# Show img test
for label in Labels:
    path = os.path.join(VAL, label)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        print(os.path.join(path, img))
        plt.imshow(img_array, cmap="gray")
        plt.title(Labels[os.listdir(path).index(img)])
        plt.show()
        break
    break

## Create data

In [ ]:
def creat_train_data(PATH):
  data = []
  images = []
  targets = []

  IMG_SIZE = tuple((config.size_img, config.size_img))
  for label in Labels:
      path = os.path.join(PATH, label)  
      index_ = Labels.index(label)
      for img in os.listdir(path):
          try:
              img_array = cv2.imread(os.path.join(path, img))
              new_array = cv2.resize(img_array, IMG_SIZE, interpolation=cv2.INTER_AREA)
              imgGray = cv2.cvtColor(new_array, cv2.COLOR_BGR2GRAY)
              imgGray_flat=np.array(imgGray).flatten()
              images.append(imgGray)
              targets.append(index_)
              data.append(imgGray_flat)
          except Exception as e:
              print(e)
  return data, images, targets

In [ ]:
Train, Images, Targets = creat_train_data(TRAIN)

In [ ]:
print(len(Images))
print(len(Train))
print(len(Targets))

In [ ]:
Val, val_images, val_targets  = creat_train_data(VAL)

In [ ]:
print(len(Val))
print(len(val_images))
print(len(val_targets))

In [ ]:
test, tes_images, test_targets  = creat_train_data(TEST)
print(len(test))
print(len(tes_images))
print(len(test_targets))

In [ ]:
val3, val3_images, val3_targets  = creat_train_data(VAL3)
print(len(val3))
print(len(val3_images))
print(len(val3_targets))

## List to array

In [ ]:
def toarray(liste):
  arr = np.array(liste)
  print(arr.shape)
  return arr

In [ ]:
# transform data train to numpy array
Images = toarray(Images)
Train = toarray(Train)
Targets = toarray(Targets)

In [ ]:

# transform val to numpy array
Val = toarray(Val)
val_images = toarray(val_images)
val_targets = toarray(val_targets)

In [ ]:
# transform val to numpy array
test = toarray(test)
tes_images = toarray(tes_images)
test_targets = toarray(test_targets)

In [ ]:
# transform val to numpy array
test = toarray(val3)
tes_images = toarray(val3_images)
test_targets = toarray(val3_targets)

In [ ]:
plt.imshow(Images[3])

## Preprocessing

In [ ]:
# facilité les calcules
def preprocess_image(img):
    return img / 255.0

In [ ]:
# preprocessing image_train
train = []
for i in range(len(Images)):
    train.append(preprocess_image(Images[i]))

In [ ]:
# Apres 
train[1]

In [ ]:
# transform train to numpy array
train = toarray(train)

In [ ]:
# preprocessing images_test
val = []
for i in range(len(val_images)):
    val.append(preprocess_image(val_images[i]))

In [ ]:
val = toarray(val)

In [ ]:
val[1]

In [ ]:
# preprocessing images_test
Test = []
for i in range(len(tes_images)):
    Test.append(preprocess_image(tes_images[i]))

In [ ]:
Test = toarray(Test)

In [ ]:
# preprocessing images_test
Val3 = []
for i in range(len(val3_images)):
    Val3.append(preprocess_image(val3_images[i]))

In [ ]:
Val3 = toarray(Val3)

In [ ]:
# Plot some image in data
for i in range(len(train[100:108])):
  plt.imshow(train[102+i].reshape(config.size_img, config.size_img), cmap="gray")
  plt.title(Labels[Targets[102+i]])
  plt.show()

## Split data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# split data 
def split(X, y):
  xtr, xtes, ytr, ytes = train_test_split(X, y, test_size=config.splitt[0], random_state=42)
  print(f'shape of train: {xtr.shape} \n shape of test: {xtes.shape} \n shape of target train: {ytr.shape} \n shape of target test: {ytes.shape}')
  return xtr, xtes, ytr, ytes

## Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(9,9))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

## Class

In [ ]:
os.makedir('Models',exist_ok=True)
!mv Models / content 

In [ ]:
# Ma fonction Main

def main():
    """
    # Fonction main
    """

    #Definition des chemins et autres variables
    # pathData = '/content/gdrive/My Drive/model4'
    # trainRatio = 0.8
    # epochs = 30
    # batch_size = 32
    # earlyStopPatience = 10


    #Definition des callbacks

    #Permet de retourner 4 metrics de suivi a chaque iteration
    csv_logger = tf.keras.callbacks.CSVLogger('/content/gdrive/My Drive/Models/log_Model4.csv', append=True, separator=',')

    #Permet de stopper l'entrainement quand le modèle n'entraine plus
    early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0, patience = config.earlyStopPatience, verbose=0, mode='auto')

    #Permet de sauvegarder le model a chaque iteration si il est meilleur que le precedent
    check = tf.keras.callbacks.ModelCheckpoint('/content/gdrive/My Drive/Models/Model4.h5', monitor='val_loss', verbose=0,
                            save_best_only=True, save_weights_only=False, mode='auto')

    #Recuperation de nos data pré traité
    # split data images
    print('-----------------------------------------------------split data images---------------------------------------------------------')
    img_train, img_test, target_train, target_test =  split(train, Targets)

    #On verifie les dimensions de nos données
    # print('DIMENSION img TRAIN ' + str(img_train.shape))
    # print('DIMENSION img TEST ' + str(img_test.shape))
    # print('DIMENSION target TRAIN ' + str(target_train.shape))
    # print('DIMENSION target TEST ' + str(target_test.shape))

    # reshape data to have a single channel
    img_train = img_train.reshape((img_train.shape[0], img_train.shape[1], img_train.shape[2], 1))
    img_test = img_test.reshape((img_test.shape[0], img_test.shape[1], img_test.shape[2], 1))

    # new dimension of data
    print('\n')
    print('-----------------------------------------------------dimension of data---------------------------------------------------------')
    print(f'img_train shape : {img_train.shape}')
    print(f'img_test shape : {img_test.shape}')

    # determiné le shape of the input images
    in_shape = img_train.shape[1:]

    # determine the number of classes
    print('\n')
    print('-----------------------------------------------------number of classes---------------------------------------------------------')
    n_classes = len(np.unique(target_train))
    print(f'Input image : {in_shape}, Les classes :{n_classes}')

    # ------------------------ On creer le modele
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(config.channels_one, kernel_size=(9, 9), activation='relu', padding="Same", strides=(1,1), kernel_initializer='he_uniform', input_shape=img_train.shape[1:] ) )
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(config.dropout))

    model.add(tf.keras.layers.Conv2D(config.channels_two, kernel_size=(3,3), padding="Same", activation='relu', strides=(1,1), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(config.dropout))

    model.add(tf.keras.layers.Conv2D(config.channels_two, kernel_size=(3,3), padding="Same", activation='relu', strides=(1,1), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(config.dropout))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(config.dense_layer_nodes, activation='relu'))
    model.add(tf.keras.layers.Dropout(config.dropout))
    model.add(tf.keras.layers.Dense(n_classes, activation = config.activation_function[0]))
    # ------------------------- END -------------------------
    
    # Add the optimizer
    # step = tf.Variable(0, trainable=False)
    # rate = tf.train.exponential_decay(0.15, step, 1, 0.9999)
    # optimizer = tf.train.AdamOptimizer(rate).minimize(cross_entropy, global_step=step)


    # On compile le modele
    model.compile(loss = tf.losses.SparseCategoricalCrossentropy(),
                  optimizer= tf.optimizers.Adamax(lr = config.learning_rate), 
                  metrics = ['accuracy'])
    
    # On lance l'entrainement du modele
    print('\n')
    print('-----------------------------------------------------Training Model---------------------------------------------------------')
    wandb.init()
    trainning = model.fit(img_train, target_train, 
                          batch_size = config.batch_size,
                          epochs = config.epochs, 
                          validation_data = (img_test, target_test))
                          # callbacks = [WandbCallback()])
                          #callbacks = [early, check,csv_logger] )
    print('\n')
    print('---------------------------------------------------------HISTORY LOSS---------------------------------------------------------------')

    loss_curve = trainning.history["loss"]
    loss_val_curve = trainning.history["val_loss"]
    plt.plot(loss_curve, label="Train")
    plt.plot(loss_val_curve, label="Val")
    plt.legend(loc='upper left')
    plt.title("Loss")
    plt.show()

    print('\n')
    print('---------------------------------------------------------HISTORY ACC---------------------------------------------------------------')
    acc_curve = trainning.history["accuracy"]
    acc_val_curve = trainning.history["val_accuracy"]

    plt.plot(acc_curve, label="Train")
    plt.plot(acc_val_curve, label="Val")
    plt.legend(loc='upper left')
    plt.title("Accuracy")
    plt.show()


    print('\n')
    print('------------------------------------------------------- TEST MODEL --------------------------------------------------------')
    loss, acc = model.evaluate(img_test, target_test)
    print("Test Loss", loss)
    print("Test Accuracy", acc)
    print('\n')

    print('----------------------------------------------------Evaluation model avec Val 1----------------------------------------------------')
    # reshape data to have a single channel
    eval = val.reshape((val.shape[0], val.shape[1], val.shape[2], 1))
    loss, acc = model.evaluate(eval, val_targets)
    print("Test Loss", loss)
    print("Test Accuracy", acc)
    print('\n')

    print('----------------------------------------------------Evaluation model avec Val 2----------------------------------------------------')
    # reshape data to have a single channel
    eval2 = Test.reshape((Test.shape[0], Test.shape[1], Test.shape[2], 1))
    loss_2, acc_2 = model.evaluate(eval2, test_targets)
    print("Test Loss", loss_2)
    print("Test Accuracy", acc_2)

    # print('----------------------------------------------------Evaluation model avec Val 3----------------------------------------------------')
    # # reshape data to have a single channel
    # eval3 = Test.reshape((Val3.shape[0], Val3.shape[1], Val3.shape[2], 1))
    # loss_3, acc_3 = model.evaluate(eval3,val3_targets )
    # print("Test Loss", loss_3)
    # print("Test Accuracy", acc_3)


    print('\n')
    print('----------------------------------------------------------CONFUSION MATRIX val1-------------------------------------------------------------------')
    Y_pred_2 = model.predict_generator(eval)
    y_pred_2 = np.argmax(Y_pred_2, axis=1)

    cm_2 = confusion_matrix(y_true=val_targets, y_pred=y_pred_2)
    cm_plot_labels = Labels
    plot_confusion_matrix(cm=cm_2, classes=cm_plot_labels, title='Confusion Matrix')

    print('\n')
    print('----------------------------------------------------------CONFUSION MATRIX val2-------------------------------------------------------------------')
    Y_pred_2_ = model.predict_generator(eval2)
    y_pred_2_ = np.argmax(Y_pred_2_, axis=1)

    cm_2_ = confusion_matrix(y_true = test_targets, y_pred = y_pred_2_)
    cm_plot_labels_ = Labels
    plot_confusion_matrix(cm = cm_2_, classes=cm_plot_labels_, title ='Confusion Matrix')
    print('\n')

    # print('----------------------------------------------------------CONFUSION MATRIX val3-------------------------------------------------------------------')
    # Y_pred_3 = model.predict_generator(eval3)
    # y_pred_3 = np.argmax(Y_pred_3, axis=1)

    # cm_3 = confusion_matrix(y_true =val3_targets , y_pred = Y_pred_3)
    # cm_plot_labels3 = Labels
    # plot_confusion_matrix(cm = cm_3, classes=cm_plot_labels3, title ='Confusion Matrix')
    # print('\n')
    print('----------------------------------------------------------Model Saving-------------------------------------------------------------------')

    model.save("/content/model_tuned.h5")

if __name__ == "__main__":
    """
    # MAIN
    """
    main()

## Check its architecture

In [ ]:
model.summary()

In [ ]:
pd.read_csv('/content/gdrive/My Drive/model4/log_Model4.csv')

# Loading the saved Model 

In [ ]:
model.save("/content/model_tuned.h5")

# loaded_model=tf.keras.models.load_model("my_model.h5")